In [ ]:
# Mount google drive
from google.colab import drive
import gc

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import openai
from openai import OpenAI

import json
import os
import io
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import time
import datetime
from random import random

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk import ne_chunk
from nltk.tree import Tree
from nltk import RegexpParser
from nltk import word_tokenize
from nltk import pos_tag

# Download the necessary NLTK data files
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
nltk.download('stopwords')
nltk.download('treebank')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker_tab.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_t

True

In [ ]:
'''
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.decomposition import PCA

from wordcloud import WordCloud

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
'''

'\nimport torch\nfrom sentence_transformers import SentenceTransformer, util\nfrom sklearn.decomposition import PCA\n\nfrom wordcloud import WordCloud\n\nimport plotly.express as px\nimport plotly.graph_objects as go\nfrom plotly.subplots import make_subplots\nimport seaborn as sns\nimport matplotlib.pyplot as plt\n\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.metrics.pairwise import cosine_similarity\nfrom sklearn.metrics import cohen_kappa_score\nfrom sklearn.metrics import confusion_matrix, classification_report\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.preprocessing import OrdinalEncoder\n'

In [ ]:
from google.colab import userdata
api_key = userdata.get('openaikey')
client = OpenAI(api_key=api_key)

In [ ]:

%cd ./drive/My Drive/SUNY_Poly_DSA598/

/content/drive/My Drive/SUNY_Poly_DSA598


In [ ]:
!ls -a

archive			    .git			    transcribe_voice_notes.ipynb
datasets		    .gitignore			    work_documents
FEVER_set_creation.ipynb    liar_gpt4omini_base_eval.ipynb
FEVER_tuning_archive.ipynb  presentation


In [ ]:

def load_jsonl(file_path, encoding='utf-8'):
    """Loads a JSON Lines file into a list of Python objects."""
    data = []
    with open(file_path, 'r', encoding=encoding) as f:  # Specify encoding for safety
        for line in f:
            data.append(json.loads(line))  # Parse each line individually
    return data

## FEVER Fine-tuning Dataset Creation

### Sentence extraction from Wikipedia text

In [ ]:
fever_path = './datasets/FEVER/'

In [ ]:
!pwd

/content/drive/My Drive/SUNY_Poly_DSA598


In [ ]:


def load_wiki_pages_minimal_memory(wiki_dir, num_files_to_load=50):
  """
  Loads Wikipedia page text from JSONL files into a dictionary (minimal memory usage).
  Loads only the 'text' field and only the first 'num_files_to_load' files.

  Args:
      wiki_dir (str): Path to the directory containing wiki-*.jsonl files.
      num_files_to_load (int): Number of wiki files to load (default: 3).

  Returns:
      dict: A dictionary where keys are page titles and values are the page text.
  """
  print(f"Attempting to load {num_files_to_load} Wikipedia pages from {wiki_dir}...")
  wiki_pages = {}
  loaded_files_count = 0
  for filename in sorted(os.listdir(wiki_dir)):
      if filename.startswith('wiki-') and filename.endswith('.jsonl'):
        filepath = os.path.join(wiki_dir, filename)
        print(f"Loading file: {filepath}")
        with open(filepath, 'r', encoding='utf-8') as f:  # Specify encoding for safety
          for line in f:
            data = json.loads(line)
            # Create a object in the dictionary for the data
            wiki_pages[data['id']] = {
              'text': data['text'],
              'lines': data['lines']
            }

        loaded_files_count += 1
      if loaded_files_count >= num_files_to_load:
            break
  return wiki_pages


wiki_pages_dir = './datasets/FEVER/wiki-pages'
num_to_load = len(os.listdir(wiki_pages_dir))
wiki_page_list_dicts = load_wiki_pages_minimal_memory(wiki_pages_dir, num_to_load)

print(f"Loaded texts from {len(wiki_page_list_dicts)} Wikipedia pages (minimal memory).")

Attempting to load 109 Wikipedia pages from ./datasets/FEVER/wiki-pages...
Loading file: ./datasets/FEVER/wiki-pages/wiki-001.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-002.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-003.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-004.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-005.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-006.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-007.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-008.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-009.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-010.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-011.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-012.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-013.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-014.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-015.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-016.jsonl
Loading file:

In [ ]:
for page, i in zip(wiki_page_list_dicts, range(1, 10)):
  print(f"Page {i}: {page}")

Page 1: 
Page 2: 1928_in_association_football
Page 3: 1986_NBA_Finals
Page 4: 1901_Villanova_Wildcats_football_team
Page 5: 1992_Northwestern_Wildcats_football_team
Page 6: 1897_Princeton_Tigers_football_team
Page 7: 1536_in_philosophy
Page 8: ...Di_terra
Page 9: 1967–68_MJHL_season


In [ ]:
fever_train = load_jsonl(fever_path + 'paper_dev.jsonl')

In [ ]:
# Print the first few  nested objects
for item, i in zip(fever_train, range(5)):
  for key in (item.keys()):
    print(key + ': ' + str(fever_train[i][key]))

id: 91198
verifiable: NOT VERIFIABLE
label: NOT ENOUGH INFO
claim: Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.
evidence: [[[108548, None, None, None]]]
id: 194462
verifiable: NOT VERIFIABLE
label: NOT ENOUGH INFO
claim: Tilda Swinton is a vegan.
evidence: [[[227768, None, None, None]]]
id: 137334
verifiable: VERIFIABLE
label: SUPPORTS
claim: Fox 2000 Pictures released the film Soul Food.
evidence: [[[289914, 283015, 'Soul_Food_-LRB-film-RRB-', 0]], [[291259, 284217, 'Soul_Food_-LRB-film-RRB-', 0]], [[293412, 285960, 'Soul_Food_-LRB-film-RRB-', 0]], [[337212, 322620, 'Soul_Food_-LRB-film-RRB-', 0]], [[337214, 322622, 'Soul_Food_-LRB-film-RRB-', 0]]]
id: 166626
verifiable: NOT VERIFIABLE
label: NOT ENOUGH INFO
claim: Anne Rice was born in New Jersey.
evidence: [[[191656, None, None, None], [191657, None, None, None]]]
id: 111897
verifiable: VERIFIABLE
label: REFUTES
claim: Telemundo is a English-language television network.

In [ ]:
len(fever_train)

9999

In [ ]:
for page_title in list(wiki_page_list_dicts.keys())[500:503]: # Inspect 3 pages
    page_data = wiki_page_list_dicts[page_title]
    print(f"\nPage Title: {page_title}")
    print(f"Keys in page_data: {page_data.keys()}")
    if 'lines' in page_data:
        print(f"Type of page_data['lines']: {type(page_data['lines'])}")
        # If it's a string, print the first few lines (sentences)
        if isinstance(page_data['lines'], str):
            sentences = page_data['lines'].strip().split('\n')
            print(f"Number of sentences: {len(sentences)}")
            if sentences:
                for line in sentences[:len(sentences)]:  # Print the first 5 sentences
                    print(line)
            else:
                print("No sentences found in the 'lines' field.")
        else:
            print("The 'lines' field is not a string.")
    else:
        print("No 'lines' key found in page_data")


Page Title: 1993_European_Cup_-LRB-athletics-RRB-
Keys in page_data: dict_keys(['text', 'lines'])
Type of page_data['lines']: <class 'str'>
Number of sentences: 5
0	The 1993 European Cup was the 14th edition of the European Cup of athletics .	European Cup	European Cup (athletics)	athletics	Athletics (sport)
1	
2	
3	The Super League Finals were held in Rome , Italy .	Rome	Rome	Italy	Italy
4

Page Title: 1905_Tempe_Normal_Owls_football_team
Keys in page_data: dict_keys(['text', 'lines'])
Type of page_data['lines']: <class 'str'>
Number of sentences: 4
0	The 1905 Tempe Normal Owls football team was an American football team that represented Tempe Normal School -LRB- later renamed Arizona State University -RRB- as an independent during the 1905 college football season .	American football	American football	Arizona State University	Arizona State University	1905 college football season	1905 college football season
1	In their seventh season under head coach Frederick M. Irish , the Owls compi

In [ ]:
fever_train = load_jsonl(fever_path + 'paper_dev.jsonl')

In [ ]:
# Print the labels in fever_train
fever_train_labels = [item['label'] for item in fever_train]
print(f"Unique labels in fever_train: {set(fever_train_labels)}")

Unique labels in fever_train: {'SUPPORTS', 'REFUTES', 'NOT ENOUGH INFO'}


In [ ]:
import random
import requests
import re
import nltk


def mediawiki_random_text(verbose=0):
    """
    Returns 5 random sentences from the introduction sections of 5 random Wikipedia articles, and the introduction text from one of the articles.
    """
    if verbose == 2:
        print("Generating random text from Wikipedia articles...")
    random_sentences = []
    intro_text = ''

    # Call the English Wikipedia API to get a random article (Category: Wikipedia articles)
    url = f"https://en.wikipedia.org/w/api.php?action=query&list=random&rnlimit=1&format=json&rnnamespace=0"
    response = requests.get(url)
    data = response.json()
    page_id = data['query']['random'][0]['id']
    page_title = data['query']['random'][0]['title']
    if verbose == 2:
        print(f"Random article: {page_title} (ID: {page_id})")
    # Call the English Wikipedia API to get the content of the article
    url = f"https://en.wikipedia.org/w/api.php?action=parse&pageid={page_id}&prop=text&format=json"
    response = requests.get(url)
    data = response.json()
    if 'parse' in data:
        page_content = data['parse']['text']['*']
        # Extract the introduction section (the first paragraph element)
        intro_section = re.search(r'<p>(.*?)</p>', page_content, re.DOTALL)
        if intro_section:
            intro_text = intro_section.group(1) # Get the text inside the first <p> tag
            # Remove HTML tags
            intro_text = re.sub(r'<.*?>', '', intro_text)
            # Remove references
            intro_text = re.sub(r'\[.*?\]', '', intro_text)
            # Remove digits that are preceded by any letter, period, colon, semicolon, endash (–), and emdash(—) and followed by a space
            intro_text = re.sub(r'(?<=[a-zA-Z0-9\.\:\;\–\—])\d+(?=\s)', '', intro_text)
            # Convert encoded html entities to unicode (e.g., &amp; to &)
            intro_text = re.sub(r'&[a-zA-Z0-9#]+;', '', intro_text)
            # Remove extra whitespace
            intro_text = re.sub(r'\s+', ' ', intro_text).strip()
            # Split into sentences
            sentences = nltk.sent_tokenize(intro_text)
            # If there are fewer than 5 sentences, recursively call the function
            if len(sentences) < 5:
                if verbose == 2:
                    print(f"Less than 5 sentences found in the introduction section of {page_title}. Calling recursively...")
                    print('_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_')
                return mediawiki_random_text(verbose=verbose)
            random_sentences = random.sample(sentences, min(5, len(sentences)))
            if verbose == 2:
                print(f"Random sentences from {page_title}:")
                for sentence in random_sentences:
                    print(f"- {sentence}")
        else:
            if verbose == 2:
                print(f"No introduction section found for {page_title}.")
    else:
        if verbose == 1:
            print(f"Failed to retrieve content for {page_title}.")
    if verbose == 1:
        print("Random text generation completed.")
        print('---------------------------------------------')
    return random_sentences, intro_text


def extract_evidence_text_debug(fever_item, wiki_page_dict, verbose=False, debug=False):
    """
    Extracts evidence sentences with debugging prints.
    """
    if verbose:
      print(f"Starting evidence extraction for claim: {fever_item['claim']}")
    if fever_item['label'] == 'NOT ENOUGH INFO':
      if verbose:
        print(f"NOT ENOUGH INFO found for claim: {fever_item['claim']}, getting random text...")
      # Call mediawiki_random_text function to get random sentences and the introduction text
      rand_sents, intro = mediawiki_random_text(verbose=0)

      return rand_sents, intro

    evidence_sentences = []
    text_str = ''
    entities = []

    # flatten the evidence set
    evidence_set = fever_item['evidence'] # This is a list of lists of lists, where each sublist contains evidence pieces that an annotator has selected for a claim
    evidence_set = [item for sublist in evidence_set for item in sublist] # Flatten the evidence set
    # Trim the first two elements of each evidence piece
    evidence_set = [evidence_piece[2:] for evidence_piece in evidence_set] # Trim the first two elements of each evidence piece
    if debug:
      print(f"DEBUG 0: Evidence set: {evidence_set}") # DEBUG 0
    # Convert inner lists to tuples before creating a set
    evidence_set = list(set(tuple(item) for item in evidence_set))
    # Convert back to list of lists (optional, based on your later usage)
    evidence_set = [list(item) for item in evidence_set]
    if debug:
      print(f"DEBUG 1: Length of evidence_set: {len(evidence_set)}\nSet: {evidence_set}") # DEBUG 0

    pages_loaded = set() # Set to keep track of loaded pages
    for evidence_piece in evidence_set: # Looping over evidence pieces (there will be 1 or more, these are the evidence sentences from different pages, for the same claim)
        if debug:
          print(f"DEBUG 2: Evidence piece: {evidence_piece}") # DEBUG 1
        if len(evidence_piece) == 2:  # Check if the evidence piece has 4 elements (annotation_id, evidence_id, page_title, sentence_id)
            page_title, sentence_id = evidence_piece # Unpack the evidence piece
            if debug:
              print(f"DEBUG 2.1: Processing evidence: page_title={page_title}, sentence_id={sentence_id}") # DEBUG 2
            if page_title is not None and sentence_id is not None: # Check if page_title and sentence_id are not None
                wiki_page = wiki_page_dict.get(page_title) # Retrieve the wiki page from the dictionary
                if debug:
                  print(f"DEBUG 3: Wiki page retrieved: {wiki_page is not None}") # DEBUG 3
                if wiki_page: # Check if the wiki page for these sentences
                    if debug:
                      print(f"DEBUG 4: Wiki page keys: {wiki_page.keys()}") # DEBUG 4
                    if 'lines' in wiki_page and isinstance(wiki_page['lines'], str): # Check if 'lines' key exists and is a string
                        lines_str = wiki_page['lines']
                        # Check if page has already been loaded and added to the text_str
                        if page_title not in pages_loaded:
                            pages_loaded.add(page_title)
                            text_str += "\n" + wiki_page['text']
                        elif debug:
                            print(f"DEBUG 4.1: Page {page_title} already loaded, skipping text addition.") # DEBUG 4.1
                        sentences = lines_str.strip().split('\n')
                        if debug:
                          print(f"DEBUG 5: Number of sentences found: {len(sentences)}") # DEBUG 5
                        if sentence_id < len(sentences):  # Check if sentence_id is within the range of sentences
                          for sentence, _ in zip(sentences, range(len(sentences))): # Loop over the sentences and their indices
                            if len(sentence.split('\t')) < 2 or sentence.split('\t')[1].strip() == '': #
                              if debug:
                                print(f"DEBUG 6: Skipping blank line at index {_}") # DEBUG 6
                              continue
                            if debug:
                              print(f"DEBUG 7: Line retrieved: {sentence} with ID: " +  sentence.split('\t')[0].strip()) # DEBUG 7
                            if int(sentence.split('\t')[0].strip()) == sentence_id:
                              sentence_text = sentence.split('\t')[1].strip() # Extract the text after the tab character (To skip the index at the beginning and the entities after: "0	The 1905 Tempe Normal Owls football team was an American football team that represented Tempe Normal School -LRB- later renamed Arizona State University -RRB- as an independent during the 1905 college football season .	American football	American football	Arizona State University	Arizona State University	1905 college football season	1905 college football season")
                              if debug:
                                print(f"DEBUG 8: Sentence text for index {sentence_id} / " + sentence.split('\t')[0].strip() + f" extracted: {sentence_text}") # DEBUG 8
                              evidence_sentences.append(sentence_text) # Append the sentence text to the evidence_sentences list
                            #else:
                                #if verbose:
                                  #print(f"\tWarning: Sentence index does not match for page: {page_title}, sentence_id: {sentence_id}")
                        else:
                          if verbose:
                            print(f"\tWarning: Sentence index out of range for page: {page_title}, sentence_id: {sentence_id} (Number of sentences: {len(sentences)})")
                    else:
                      if verbose:
                        print(f"\tWarning: Could not retrieve page or lines for title: {page_title}")
                else:
                  if verbose:
                    print(f"\tWarning: Could not retrieve wiki page for title: {page_title}")
        else:
          if verbose:
            print(f"\tWarning: Unexpected evidence format: {evidence_piece}")
        if verbose:
          print(f"DEBUG 9: Evidence sentences for page {page_title} have been extracted.")
          print("_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")
    if verbose:
      print(f"Evidence extraction completed for claim: {fever_item['claim']}")
      print(f"Number of evidence sentences found: {len(evidence_sentences)}")
      print('--------------------------------------------------------------')
    return evidence_sentences, text_str


In [ ]:
# Test the mediawiki_random_text function
random_text, intro_text = mediawiki_random_text(verbose=True)
print("Random text from Wikipedia:")
print(random_text)
print("Introduction text:")
print(intro_text)

In [ ]:
# Test run evidence extraction
claim_item = fever_train[25]
evidence, full_text = extract_evidence_text_debug(claim_item, wiki_page_list_dicts, verbose=True, debug=True)

print(f"\nClaim: {claim_item['claim']}")
print("\nEvidence Sentences:")
for sentence in evidence:
    print(f"- {sentence}")
print("\nFull Text:")
print(full_text)

Starting evidence extraction for claim: Janet Leigh was incapable of writing.
DEBUG 0: Evidence set: [['Janet_Leigh', 0], ['Janet_Leigh', 12], ['Janet_Leigh', 0], ['Janet_Leigh', 12], ['Janet_Leigh', 0], ['Janet_Leigh', 0], ['Janet_Leigh', 0]]
DEBUG 1: Length of evidence_set: 2
Set: [['Janet_Leigh', 0], ['Janet_Leigh', 12]]
DEBUG 2: Evidence piece: ['Janet_Leigh', 0]
DEBUG 2.1: Processing evidence: page_title=Janet_Leigh, sentence_id=0
DEBUG 3: Wiki page retrieved: True
DEBUG 4: Wiki page keys: dict_keys(['text', 'lines'])
DEBUG 5: Number of sentences found: 18
DEBUG 7: Line retrieved: 0	Janet Leigh -LRB- born Jeanette Helen Morrison ; July 6 , 1927 -- October 3 , 2004 -RRB- was an American actress , singer , dancer and author . with ID: 0
DEBUG 8: Sentence text for index 0 / 0 extracted: Janet Leigh -LRB- born Jeanette Helen Morrison ; July 6 , 1927 -- October 3 , 2004 -RRB- was an American actress , singer , dancer and author .
DEBUG 7: Line retrieved: 1	She is best remembered for he

### Create training files for Various Models

In [ ]:
# A for loop to print the time every 30 seconds
for i in range(0, 3600):
  print(f"Time: {datetime.datetime.now().strftime('%H:%M:%S')}")
  time.sleep(30)

Time: 02:36:23
Time: 02:36:53
Time: 02:37:23
Time: 02:37:53
Time: 02:38:23


In [ ]:
gc.collect()

77

In [ ]:
for fever_item in fever_data[0:10]:
  print(fever_item)


{'id': 91198, 'verifiable': 'NOT VERIFIABLE', 'label': 'NOT ENOUGH INFO', 'claim': 'Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.', 'evidence': [[[108548, None, None, None]]]}
{'id': 194462, 'verifiable': 'NOT VERIFIABLE', 'label': 'NOT ENOUGH INFO', 'claim': 'Tilda Swinton is a vegan.', 'evidence': [[[227768, None, None, None]]]}
{'id': 137334, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'Fox 2000 Pictures released the film Soul Food.', 'evidence': [[[289914, 283015, 'Soul_Food_-LRB-film-RRB-', 0]], [[291259, 284217, 'Soul_Food_-LRB-film-RRB-', 0]], [[293412, 285960, 'Soul_Food_-LRB-film-RRB-', 0]], [[337212, 322620, 'Soul_Food_-LRB-film-RRB-', 0]], [[337214, 322622, 'Soul_Food_-LRB-film-RRB-', 0]]]}
{'id': 166626, 'verifiable': 'NOT VERIFIABLE', 'label': 'NOT ENOUGH INFO', 'claim': 'Anne Rice was born in New Jersey.', 'evidence': [[[191656, None, None, None], [191657, None, None, None]]]}
{'id': 111897, 'ver

In [ ]:
# Set the date of the data creation (today)
now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=-5)))
date_fmt = now.strftime("%m-%d")
print(f"Today's date: {date_fmt}")

fever_label_mapping = {
    "SUPPORTS": "1",
    "REFUTES": "0",
    "NOT ENOUGH INFO": "-1"
}

# Helper function to allow balanced index-based selection (which is required to segment the data between different NLI models, ensuring maximum variability)
def intersperse_labels(data):
  """Intersperses SUPPORTS and REFUTES examples for guaranteed balance."""
  print('>>>>>> STARTING INTERSPERSING SUPPORTS AND REFUTES <<<<<<<<<<')
  supports = [item for item in data if item.get('label') == 'SUPPORTS']
  refutes = [item for item in data if item.get('label') == 'REFUTES']
  nei = [item for item in data if item.get('label') == 'NOT ENOUGH INFO']

  interspersed_data = []
  min_len = min(len(supports), len(refutes)) # Handle potential slight imbalances
  for i in range(min_len):
      interspersed_data.append(supports[i])
      interspersed_data.append(refutes[i])
      interspersed_data.append(nei[i]) # Add the NOT ENOUGH INFO item

  # print(f"Number of SUPPORTS in resulting data: {len([item for item in interspersed_data if item['label'] == 'SUPPORTS'])}")
  # print(f"Number of REFUTES in resulting data: {len([item for item in interspersed_data if item['label'] == 'REFUTES'])}")
  print(f"Length of fever_data after interspersing: {len(interspersed_data)}")
  print('>>>>>>>>>>> INTERSPERSING SUPPORTS AND REFUTES COMPLETED <<<<<<<<<<<')
  return interspersed_data

# Helper function to dry run the evidence extraction and return jsonl without the items that fail to associate with the wiki pages
def remove_items_with_no_evidence(fever_data, wiki_pages, verbose=False):
    """
    Removes items from fever_data that have no evidence associated with them before we attempt to retrieve it during the conversion.
    """
    print(f">>>>>>>>>>> STARTING REMOVAL OF ITEMS WITH NO EVIDENCE <<<<<<<<<<<")
    # Call the the extraction function in a loop and only add the items that have evidence to the new list
    filtered_data = []
    dropped_true_count = 0
    dropped_false_count = 0
    for fever_item in fever_data:
        claim = fever_item['claim']
        label = fever_item['label']

        if label == 'NOT ENOUGH INFO':
          filtered_data.append(fever_item)
          if verbose:
            print(f"\tItem is {label}, skipping evidence extraction.")
          continue
        else:
          if verbose:
            print(f"\tItem is {label}, previewing evidence extraction...")
          evidence_sentences, _ = extract_evidence_text_debug(fever_item, wiki_pages)
          if len(evidence_sentences) > 0:
            filtered_data.append(fever_item)
          else:
            if label == 'SUPPORTS':
                dropped_true_count += 1
            elif label == 'REFUTES':
                dropped_false_count += 1
            if verbose:
              print(f"\tWarning: Dropping item with claim: {claim} due to no evidence found.")
              continue

    print(f"Dropped {dropped_true_count} SUPPORTS and {dropped_false_count} REFUTES items due to no evidence found.")
    print(f"Total items dropped: {dropped_true_count + dropped_false_count}")
    print(f"Total items remaining: {len(filtered_data)}")
    print('>>>>>>>>>>> REMOVAL OF ITEMS WITH NO EVIDENCE COMPLETED <<<<<<<<<<<')
    print('---------------------------------------------')
    return filtered_data


Today's date: 03-29


In [ ]:
# Get the current time in 24 hour time, hours:minutes:seconds
current_time = datetime.datetime.now().strftime("%H:%M:%S")
# Print the seconds only
seconds = current_time.split(':')[-1]
print("22395613294602193653092643591633490623395613304603")
print(seconds)



395613294602193653092643591633490623395613304603
22


In [ ]:

def convert_fever_to_fine_tuning(fever_data, output_filepath, wiki_page_dict, user_prompt, sys_prompt, mode='fact_checking', data_type='tabular', train_size=200, valid_size=60, data_segmentation=False, debug=False):


    # Initialize jsonl object lists for 'JSONL' data type
    train_jsonl = []
    valid_jsonl = []
    # Initialize dataframes for 'tabular' data type
    if mode == 'evidence_extraction':
        train_df = pd.DataFrame(columns=['claim', 'evidence_sentences', 'full_text', 'label'])
        valid_df = pd.DataFrame(columns=['claim', 'evidence_sentences', 'full_text', 'label'])
    elif mode == 'fact_checking':
        train_df = pd.DataFrame(columns=['claim', 'evidence_sentences', 'label'])
        valid_df = pd.DataFrame(columns=['claim', 'evidence_sentences', 'label'])


    print(f"########################## PREPROCESSING STARTED ##########################")

    ####### THE NEXT TWO STEPS ARE FOR CREATING A SET WITH ONLY SUPPORTS AND REFUTES #######
    '''
    # Remove items where there is not enough info
    print(f"Length of fever_data before filtering out NOT ENOUGH INFO: {len(fever_data)}")
    fever_data = [item for item in fever_data if item['label'] != 'NOT ENOUGH INFO']
    print(f"Length of fever_data after filtering out NOT ENOUGH INFO: {len(fever_data)}")

    # Remove items where there is no evidence
    fever_data = [item for item in fever_data if len(item['evidence']) > 0]
    print(f"Length of fever_data after filtering out items with no evidence: {len(fever_data)}")
    print(f"Number of SUPPORTS in fever_data: {len([item for item in fever_data if item['label'] == 'SUPPORTS'])}")
    print(f"Number of REFUTES in fever_data: {len([item for item in fever_data if item['label'] == 'REFUTES'])}")
    print('---------------------------------------------')
    '''

    # Remove items where there is no wiki page associated with the claim
    print(f"Length of fever_data before filtering out items with no wiki page: {len(fever_data)}")
    fever_data = remove_items_with_no_evidence(fever_data, wiki_page_dict)
    print(f"Length of fever_data after filtering out items with no wiki page: {len(fever_data)}")

    # Reorder the data by interspersing SUPPORTS and REFUTES (one item and then the other), and balance the data
    fever_data = intersperse_labels(fever_data)

    # Check the number of SUPPORTS and REFUTES in the data and reduce the number of NOT ENOUGH INFO items to the same number
   # max_label_count = min(len([item for item in fever_data if item['label'] == 'SUPPORTS']), len([item for item in fever_data if item['label'] == 'REFUTES']))
    # Reduce the number of NOT ENOUGH INFO items to the same number as the max label count
    #fever_data['NOT ENOUGH INFO'] = fever_data['NOT ENOUGH INFO'][:max_label_count]
    print(f"Number of SUPPORTS in fever_data: {len([item for item in fever_data if item['label'] == 'SUPPORTS'])}")
    print(f"Number of REFUTES in fever_data: {len([item for item in fever_data if item['label'] == 'REFUTES'])}")
    print(f"Number of NOT ENOUGH INFO in fever_data: {len([item for item in fever_data if item['label'] == 'NOT ENOUGH INFO'])}")

    print(f"########################## PREPROCESSING COMPLETE ##########################")
    print('---------------------------------------------')


    if data_segmentation and (train_size + valid_size) > (len(fever_data) // 2):
        print(f"Warning: Data segmentation is selected but the sum of train_size and valid_size exceeds half of the fever data size. Adjusting to max size with a 80-20 split.")
        train_size = (len(fever_data) // 2) * 0.8
        valid_size = (len(fever_data) // 2) * 0.2
    elif not data_segmentation and (train_size + valid_size) > len(fever_data):
        print(f"Warning: Data segmentation is not selected but the sum of train_size and valid_size exceeds the fever data size. Adjusting to max size with a 80-20 split.")
        train_size = len(fever_data) * 0.8
        valid_size = len(fever_data) * 0.2

    if mode == 'evidence_extraction':
        print('############################# START EVIDENCE EXTRACTION ###################')
        print(f"Running conversion for evidence extraction mode...")
        print('---------------------------------------------')

        # If data segmentation is enabled, use only the top half of the fever data, because the bottom half is used for the fact checking mode (max train + valid size = len(fever_data) // 2)
        if data_segmentation:
            print(f"Data segmentation enabled. Using top half of the dataset for evidence extraction.")
            train_data = fever_data[:train_size]
            print(f"Getting data for training from 0 to {train_size}")
            valid_data = fever_data[train_size:train_size + valid_size]
            print(f"Getting data for validation from {train_size} to {train_size + valid_size}")
        # If data segmentation is not enabled, use the whole fever data
        else:
          # Subset and balance the training data
          train_data = fever_data[:train_size]
          valid_data = fever_data[train_size:train_size + valid_size]

        # Print the number of claims in each subset
        print(f"Number of claims in training data before processing: {len(train_data)}")
        print(f"Number of claims in validation data before processing: {len(valid_data)}")

        # Process Training Data
        skipped_count = 0
        for fever_item in train_data:
          claim = fever_item['claim']
          if debug:
            print(f"Processing claim: {claim}")
          evidence_sentences, full_text = extract_evidence_text_debug(fever_item, wiki_page_dict, verbose=True)

          if len(evidence_sentences) > 0:
            evidence_sentences = "\n".join(set(evidence_sentences))
            if data_type == 'tabular':
              # Add the row to the dataframe
              train_df.loc[len(train_df)] = [claim, evidence_sentences, full_text, fever_item['label']]
              if debug:
                print(f"DEBUG (TABULAR): Evidence sentences for training data: {evidence_sentences}")
            elif data_type == 'jsonl':
              usr_prompt = user_prompt + f"Claim:\n{claim}\n\nSource Text:\n{full_text}"
              if debug:
                print(f"DEBUG (JSONL): Evidence sentences for training data: {evidence_sentences}")
                print(f"DEBUG (JSONL): User prompt for training data: {usr_prompt}")
              completion = evidence_sentences
              train_jsonl.append({"messages": [{"role": "system", "content": sys_prompt}, {"role": "user", "content": usr_prompt}, {"role": "assistant", "content": completion}]})
          else:
            skipped_count += 1
            print(f"WARNING: In assembling the training data, no evidence found for claim: {claim} with label: {fever_item['label']}")
            continue
        print(f"Skipped claims in training data due to no evidence: {skipped_count}")
        print('---------------------------------------------')

        # Process Validation Data
        skipped_count = 0
        for fever_item in valid_data:
          claim = fever_item['claim']
          evidence_sentences, full_text = extract_evidence_text_debug(fever_item, wiki_page_dict, verbose=True)

          if len(evidence_sentences) > 0:
            evidence_sentences = "\n".join(set(evidence_sentences))
            if data_type == 'tabular':
              valid_df.loc[len(valid_df)] = [claim, evidence_sentences, full_text, fever_item['label']]
            elif data_type == 'jsonl':
              usr_prompt = user_prompt + f"Claim:\n{claim}\n\nSource Text:\n{full_text}"
              completion = evidence_sentences
              valid_jsonl.append({"messages": [{"role": "system", "content": sys_prompt}, {"role": "user", "content": usr_prompt}, {"role": "assistant", "content": completion}]})
          else:
            skipped_count += 1
            print(f"WARNING: In assembling the validation data, no evidence found for claim: {claim} with label: {fever_item['label']}")
            continue
        print(f"Skipped claims in validation data due to no evidence: {skipped_count}")
        print('---------------------------------------------')


        if data_type == 'tabular':
            train_run_count = len(os.listdir(f'{fever_path}/tabular_sentEx_paper_dev_train'))
            valid_run_count = len(os.listdir(f'{fever_path}/tabular_sentEx_paper_dev_valid'))
            print(f"Number of files in tabular_sentEx_paper_dev_train: {train_run_count}")
            print(f"Number of files in tabular_sentEx_paper_dev_valid: {valid_run_count}")
            train_run_count = str(train_run_count).zfill(3)
            valid_run_count = str(valid_run_count).zfill(3)

            print('---------------------------------------------')
            print(f"Length of the evidence extraction training data: {len(train_df)}")
            print(f"Number of SUPPORTS in the training data: {len([item for item in train_df['label'] if item == 'SUPPORTS'])}")
            print(f"Number of REFUTES in the training data: {len([item for item in train_df['label'] if item == 'REFUTES'])}")
            print(f'Number of NOT ENOUGH INFO in the training data: {len([item for item in train_df["label"] if item == "NOT ENOUGH INFO"])}')
            print('---------------------------------------------')
            print(f"Length of the evidence extraction validation data: {len(valid_df)}")
            print(f"Number of SUPPORTS in the validation data: {len([item for item in valid_df['label'] if item == 'SUPPORTS'])}")
            print(f"Number of REFUTES in the validation data: {len([item for item in valid_df['label'] if item == 'REFUTES'])}")
            print(f'Number of NOT ENOUGH INFO in the validation data: {len([item for item in valid_df["label"] if item == "NOT ENOUGH INFO"])}')
            print('---------------------------------------------')

            # Save the dataframes as CSV files
            train_df.to_csv(f"{fever_path}tabular_sentEx_paper_dev_train/{output_filepath}{train_size}_{date_fmt}_{train_run_count}.csv", index=False)
            valid_df.to_csv(f"{fever_path}tabular_sentEx_paper_dev_valid/{output_filepath}{valid_size}_{date_fmt}_{valid_run_count}.csv", index=False)

            print(f"FEVER data conversion to tabular format completed. Output files:\n - {fever_path}tabular_sentEx_paper_dev_train/{output_filepath}{train_size}_{date_fmt}_{train_run_count}.csv\n - {fever_path}tabular_sentEx_paper_dev_valid/{output_filepath}{valid_size}_{date_fmt}_{valid_run_count}.csv")

        elif data_type == 'jsonl':

          train_run_count = len(os.listdir(f'{fever_path}/GPT_sentEx_paper_dev_train'))
          valid_run_count = len(os.listdir(f'{fever_path}/GPT_sentEx_paper_dev_valid'))
          print(f"Number of files in GPT_sentEx_paper_dev_train: {train_run_count}")
          print(f"Number of files in GPT_sentEx_paper_dev_valid: {valid_run_count}")
          train_run_count = str(train_run_count).zfill(3)
          valid_run_count = str(valid_run_count).zfill(3)

          print('---------------------------------------------')
          print(f"Length of the evidence extraction training data: {len(train_jsonl)}")
          print(f"Number of SUPPORTS in the training data: {len([item for item in train_data if item['label'] == 'SUPPORTS'])}")
          print(f"Number of REFUTES in the training data: {len([item for item in train_data if item['label'] == 'REFUTES'])}")
          print(f'Number of NOT ENOUGH INFO in the training data: {len([item for item in train_data if item["label"] == "NOT ENOUGH INFO"])}')
          print('---------------------------------------------')
          print(f"Length of the evidence extraction validation data: {len(valid_jsonl)}")
          print(f"Number of SUPPORTS in the validation data: {len([item for item in valid_data if item['label'] == 'SUPPORTS'])}")
          print(f"Number of REFUTES in the validation data: {len([item for item in valid_data if item['label'] == 'REFUTES'])}")
          print(f'Number of NOT ENOUGH INFO in the validation data: {len([item for item in valid_data if item["label"] == "NOT ENOUGH INFO"])}')
          print('---------------------------------------------')

          # Save the jsonl object as jsonlines files
          with open(f"{fever_path}GPT_sentEx_paper_dev_train/{output_filepath}{train_size}_{date_fmt}_{train_run_count}.jsonl", 'w') as f:
            for data_item in train_jsonl:
                  json.dump(data_item, f)
                  f.write('\n')
          with open(f"{fever_path}GPT_sentEx_paper_dev_valid/{output_filepath}{valid_size}_{date_fmt}_{valid_run_count}.jsonl", 'w') as f:
            for data_item in valid_jsonl:
                  json.dump(data_item, f)
                  f.write('\n')
          print(f"FEVER data conversion to OpenAI JSONL completed. Output files:\n - {fever_path}GPT_sentEx_paper_dev_train/{output_filepath}{train_size}_{date_fmt}_{train_run_count}.jsonl\n - {fever_path}GPT_sentEx_paper_dev_valid/{output_filepath}{valid_size}_{date_fmt}_{valid_run_count}.jsonl")
        print('################### END OF CONVERSION FOR EVIDENCE EXTRACTION ###################\n')

    ############### END OF EVIDENCE EXTRACTION MODE ################

    ############### START OF FACT CHECKING MODE ###################
    elif mode == 'fact_checking':
        print('######################### START FACT CHECKING ###########################')
        print(f"Running conversion for fact checking mode...")
        print('---------------------------------------------')

        # If data segmentation is enabled, use only the bottom half of the fever data, because the top half is used for the evidence extraction mode (max train + valid size = len(fever_data) // 2)
        if data_segmentation:
            print(f"Data segmentation enabled. Using bottom half of the dataset for fact checking.")
            train_data = fever_data[len(fever_data) // 2:(len(fever_data) // 2) + train_size]
            print(f"Getting data for training from {len(fever_data) // 2} to {len(fever_data) // 2 + train_size}")
            valid_data = fever_data[(len(fever_data) // 2) + train_size:len(fever_data) // 2 + train_size + valid_size]
            print(f"Getting data for validation from {len(fever_data) // 2 + train_size} to {len(fever_data) // 2 + train_size + valid_size}")
        # If data segmentation is not enabled, use the whole fever data
        else:
          # Subset and balance the training data
          train_data = fever_data[:train_size]
          valid_data = fever_data[train_size:train_size + valid_size]

        # Print the number of claims in each subset
        print(f"Number of claims in training data before processing: {len(train_data)}")
        print(f"Number of claims in validation data before processing: {len(valid_data)}")

        # Process Training data
        skipped_count = 0
        for fever_item in train_data:
          claim = fever_item['claim']
          if debug:
            print(f"Processing claim: {claim}")
          fever_label = fever_item['label']
          mapped_label = fever_label_mapping.get(fever_label)
          evidence_sentences, full_text = extract_evidence_text_debug(fever_item, wiki_page_dict, verbose=True)

          if mapped_label:
            if len(evidence_sentences) > 0:
              evidence_sentences = "\n".join(set(evidence_sentences)) # Join the sentences
              if data_type == 'tabular':
                train_df.loc[len(train_df)] = [claim, evidence_sentences, fever_item['label']]
                if debug:
                  print(f"DEBUG (TABULAR): Evidence sentences for training data: {evidence_sentences}")
              elif data_type == 'jsonl':
                usr_prompt = user_prompt + f"Claim:\n{claim}\n\nEvidence sentences:\n{evidence_sentences}"
                if debug:
                  print(f"Evidence sentences: {evidence_sentences}")
                  print(f"User prompt: {usr_prompt}")
                completion = mapped_label
                train_jsonl.append({"messages": [{"role": "system", "content": sys_prompt}, {"role": "user", "content": usr_prompt}, {"role": "assistant", "content": completion}]})
            else:
              skipped_count += 1
              print(f"WARNING: In assembling the training data, no evidence found for claim: {claim} with label: {fever_item['label']}")
              continue
        print(f"Skipped claims in training data due to no evidence: {skipped_count}")
        print('----------------------------------------------')

        # Process Validation Data
        skipped_count = 0
        for fever_item in valid_data:
          claim = fever_item['claim']
          fever_label = fever_item['label']
          mapped_label = fever_label_mapping.get(fever_label)
          evidence_sentences, full_text = extract_evidence_text_debug(fever_item, wiki_page_dict, verbose=True)

          if mapped_label:
            if len(evidence_sentences) > 0:
              evidence_sentences = "\n".join(evidence_sentences) # Join the sentences
              if data_type == 'tabular':
                valid_df.loc[len(valid_df)] = [claim, evidence_sentences, fever_item['label']]
              elif data_type == 'jsonl':
                usr_prompt = user_prompt + f"Claim:\n{claim}\n\nEvidence:\n{evidence_sentences}"
                completion = mapped_label
                valid_jsonl.append({"messages": [{"role": "system", "content": sys_prompt}, {"role": "user", "content": usr_prompt}, {"role": "assistant", "content": completion}]})
            else:
                skipped_count += 1
                print(f"WARNING: In assembling the validation data, no evidence found for claim: {claim} with label: {fever_item['label']}")
                continue
        print(f"Skipped claims in validation data due to no evidence: {skipped_count}")
        print('----------------------------------------------')

        if data_type == 'tabular':
            train_run_count = len(os.listdir(f'{fever_path}/tabular_clf_paper_dev_train'))
            valid_run_count = len(os.listdir(f'{fever_path}/tabular_clf_paper_dev_valid'))
            print(f"Number of files in tabular_clf_paper_dev_train: {train_run_count}")
            print(f"Number of files in tabular_clf_paper_dev_valid: {valid_run_count}")
            train_run_count = str(train_run_count).zfill(3)
            valid_run_count = str(valid_run_count).zfill(3)

            print('----------------------------------------------')
            print(f"Length of the fact checking training data: {len(train_df)}")
            print(f"Number of SUPPORTS in the training data: {len([item for item in train_df['label'] if item == 'SUPPORTS'])}")
            print(f"Number of REFUTES in the training data: {len([item for item in train_df['label'] if item == 'REFUTES'])}")
            print(f'Number of NOT ENOUGH INFO in the training data: {len([item for item in train_df["label"] if item == "NOT ENOUGH INFO"])}')
            print('----------------------------------------------')
            print(f"Length of the fact checking validation data: {len(valid_df)}")
            print(f"Number of SUPPORTS in the validation data: {len([item for item in valid_df['label'] if item == 'SUPPORTS'])}")
            print(f"Number of REFUTES in the validation data: {len([item for item in valid_df['label'] if item == 'REFUTES'])}")
            print(f'Number of NOT ENOUGH INFO in the validation data: {len([item for item in valid_df["label"] if item == "NOT ENOUGH INFO"])}')
            print('----------------------------------------------')

            # Save the dataframes as CSV files
            train_df.to_csv(f"{fever_path}tabular_clf_paper_dev_train/{output_filepath}{train_size}_{date_fmt}_{train_run_count}.csv", index=False)
            valid_df.to_csv(f"{fever_path}tabular_clf_paper_dev_valid/{output_filepath}{valid_size}_{date_fmt}_{valid_run_count}.csv", index=False)

            print(f"FEVER data conversion to tabular format completed. Output files:\n - {fever_path}tabular_clf_paper_dev_train/{output_filepath}{train_size}_{date_fmt}_{train_run_count}.csv\n - {fever_path}tabular_clf_paper_dev_valid/{output_filepath}{valid_size}_{date_fmt}_{valid_run_count}.csv")

        elif data_type == 'jsonl':
          train_run_count = len(os.listdir(f'{fever_path}/GPT_clf_paper_dev_train'))
          valid_run_count = len(os.listdir(f'{fever_path}/GPT_clf_paper_dev_valid'))
          print(f"Number of files in GPT_clf_paper_dev_train: {train_run_count}")
          print(f"Number of files in GPT_clf_paper_dev_valid: {valid_run_count}")
          train_run_count = str(train_run_count).zfill(3)
          valid_run_count = str(valid_run_count).zfill(3)

          print('----------------------------------------------')
          print(f"Length of the fact checking training data: {len(train_jsonl)}")
          print(f"Number of SUPPORTS in the training data: {len([item for item in train_data if item['label'] == 'SUPPORTS'])}")
          print(f"Number of REFUTES in the training data: {len([item for item in train_data if item['label'] == 'REFUTES'])}")
          print(f'Number of NOT ENOUGH INFO in the training data: {len([item for item in train_data if item["label"] == "NOT ENOUGH INFO"])}')
          print('----------------------------------------------')
          print(f"Length of the fact checking validation data: {len(valid_jsonl)}")
          print(f"Number of SUPPORTS in the validation data: {len([item for item in valid_data if item['label'] == 'SUPPORTS'])}")
          print(f"Number of REFUTES in the validation data: {len([item for item in valid_data if item['label'] == 'REFUTES'])}")
          print(f'Number of NOT ENOUGH INFO in the validation data: {len([item for item in valid_data if item["label"] == "NOT ENOUGH INFO"])}')
          print('----------------------------------------------')

          with open(f"{fever_path}tabular_clf_paper_dev_train/{output_filepath}{train_size}_{date_fmt}_{train_run_count}.jsonl", 'w') as f:
            for data_item in train_jsonl:
                  json.dump(data_item, f)
                  f.write('\n')
          with open(f"{fever_path}tabular_clf_paper_dev_valid/{output_filepath}{valid_size}_{date_fmt}_{valid_run_count}.jsonl", 'w') as f:
            for data_item in valid_jsonl:
                  json.dump(data_item, f)
                  f.write('\n')
          print(f"FEVER data conversion to OpenAI JSONL completed. Output files:\n - {fever_path}GPT_clf_paper_dev_train/{output_filepath}{train_size}_{date_fmt}_{train_run_count}.jsonl\n - {fever_path}GPT_clf_paper_dev_valid/{output_filepath}{valid_size}_{date_fmt}_{valid_run_count}.jsonl")
        print('################### END OF CONVERSION FOR FACT CHECKING ###################\n')


In [ ]:
fever_train_filepath = fever_path + 'paper_dev.jsonl'
fever_data = load_jsonl(fever_train_filepath, 'utf-8')

########## STUDY VARIABLES – SYSTEM AND USER PROMPTS AND TRAINING AND VALIDATION SET SIZES #########
sentEx_sys_prompt = ""
sentEx_user_prompt = f"Extract sentences from the source text that are relevant (either supporting or refuting) to the preceding claim. Return a comma separated list of sentences.\n\n"
clf_sys_prompt = ""
clf_user_prompt = "Classify the truthfulness of the text out of the following categories: '1' (if the claim is true), '0' (if the claim is false), given the evidence. Do not use any other labels.\n\n"
# MAXIMUM POSSIBLE SIZE FROM PAPER_DEV IF SEGMENTING BETWEEN SENTENCE EXTRACTION AND FACT-CHECKING: 4959 SAMPLES (per the removal of 50 SUPPORTS and REFUTES samples that lack a wiki page and thus evidence)
# NEAREST MAXIMUM TRAIN SIZE (that is a multiple of 3 and 70% of the segment): 3468
# NEAREST MAXIMUM VALIDATION SIZE (that is a multiple of 3 and 30% of the segment): 1488
train_n = 3468 # Should be a multiple of 3
valid_n = 1488 # Should be a multiple of 3
####################################################################################################

output_filepath = 'v1_segmented_n' # Subfolder + filename prefix (e.g., 'GPT_clf_paper_dev_train/prompt_v1_segmented_n'). If no subfolder is specified files will be saved to FEVER parent folder.

convert_fever_to_fine_tuning(fever_data, output_filepath, wiki_page_list_dicts, sentEx_user_prompt, sentEx_sys_prompt, 'evidence_extraction', train_size=train_n, valid_size=valid_n, data_segmentation=True)
convert_fever_to_fine_tuning(fever_data, output_filepath, wiki_page_list_dicts, clf_user_prompt, clf_sys_prompt, 'fact_checking', train_size=train_n, valid_size=valid_n, data_segmentation=True)

Streaming output truncated to the last 5000 lines.
Starting evidence extraction for claim: Reign Over Me is an American movie.
DEBUG 9: Evidence sentences for page Reign_Over_Me have been extracted.
_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
Evidence extraction completed for claim: Reign Over Me is an American movie.
Number of evidence sentences found: 1
--------------------------------------------------------------
Starting evidence extraction for claim: Middle-earth was created by J. K. Rowling.
DEBUG 9: Evidence sentences for page Middle-earth have been extracted.
_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
Evidence extraction completed for claim: Middle-earth was created by J. K. Rowling.
Number of evidence sentences found: 1
--------------------------------------------------------------
Starting evidence extraction for claim: Shane McMahon never officially retired.
NOT ENOUGH INFO found for claim: Shane McMahon never officially retired., getting random text...
Start

In [ ]:
# Print the first several rows of each of the two training files
sentEx_train = load_jsonl(f'{fever_path}GPT_sentEx_paper_dev_train/prompt_v1_segmented_n100_03-16_000.jsonl')
sentEx_valid = load_jsonl(f'{fever_path}GPT_sentEx_paper_dev_valid/prompt_v1_segmented_n30_03-16_000.jsonl')
clf_train = load_jsonl(f'{fever_path}GPT_clf_paper_dev_train/prompt_v1_segmented_n100_03-16_000.jsonl')
clf_valid = load_jsonl(f'{fever_path}GPT_clf_paper_dev_valid/prompt_v1_segmented_n30_03-16_000.jsonl')

# Print the first few  nested objects
print(f"Classifiction training data:")
for item, i in zip(clf_train, range(5)):
  for key in (item.keys()):
    print(key + ': ' + str(clf_train[i][key]))
print('-------------------------------------------------------------------------------------------')
print(f"Classifiction validation data:")
for item, i in zip(clf_valid, range(5)):
  for key in (item.keys()):
    print(key + ': ' + str(clf_valid[i][key]))
print('-------------------------------------------------------------------------------------------')
print(f"Evidence extraction training data:")
for item, i in zip(sentEx_train, range(5)):
  for key in (item.keys()):
    print(key + ': ' + str(sentEx_train[i][key]))
print('-------------------------------------------------------------------------------------------')
print(f"Evidence extraction validation data:")
for item, i in zip(sentEx_valid, range(5)):
  for key in (item.keys()):
    print(key + ': ' + str(sentEx_valid[i][key]))
print('-------------------------------------------------------------------------------------------')





Classifiction training data:
messages: [{'role': 'system', 'content': ''}, {'role': 'user', 'content': "Classify the truthfulness of the text out of the following categories: '1' (if the claim is true), '0' (if the claim is false), given the evidence. Do not use any other labels.\n\nClaim: Riddick is in a science fiction film.\n\nEvidence:\nRiddick is a Furyan , a member of a warrior race obliterated by a military campaign that left Furya desolate , and is one of the last of his kind .\nRiddick was once a mercenary , then part of a security force , and later a soldier .\nActor Vin Diesel has played the title role in all of the Riddick-based films and video games so far .\nRichard B. Riddick , more commonly known as Riddick , is a fictional character and the antihero of four films in the Riddick series -LRB- Pitch Black , The Chronicles of Riddick , the animated movie The Chronicles of Riddick : Dark Fury , and Riddick -RRB- , as well as the two video games The Chronicles of Riddick : E

In [ ]:
# Collect metadata about the files (dataset_name  avg_evdc_len  max_evdc_len  min_evdc_len  sys_msg user_msg  sys_msg_num_tokens  sys_msg_avg_wrd_len usr_msg_num_tokens	usr_msg_avg_wrd_len)

# Helper function to tokenize text and return number of tokens and semantic complexity
def tokenize_text(text):
    """
    Tokenizes the input text and returns the number of tokens and semantic complexity.
    """
    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Calculate the number of tokens
    num_tokens = len(tokens)

    # Calculate semantic complexity using a simple heuristic (e.g., average word length)
    avg_word_length = sum(len(word) for word in tokens) / num_tokens if num_tokens > 0 else 0

    return num_tokens, avg_word_length


# Create a function to collect metadata about the files
def collect_metadata(in_path, out_path, mode):
    """
    Collects metadata about the file in the input path and saves it to a CSV file in the output path.
    """
    metadata = pd.DataFrame(columns=['dataset_name', 'size', 'avg_evdc_len', 'max_evdc_len', 'min_evdc_len', 'avg_evdc_tkns', 'avg_claim_tkns', 'max_claim_tkns', 'min_claim_tkns', 'sys_msg', 'user_msg', 'sys_msg_num_tokens', 'sys_msg_avg_wrd_len', 'usr_msg_num_tokens', 'usr_msg_avg_wrd_len'])
    filename = os.path.basename(in_path)
    print(f"Processing file: {filename}")
    set_id = filename.split('_')[-1]
    if filename.endswith('.jsonl'):
        with open(in_path, 'r', encoding='utf-8') as f:
            data = [json.loads(line) for line in f]
            ds_size = len(data)
            # Calculate average and max evidence sentence length
            evdc_counts = []
            evdc_tokens = []
            user_stub_tokens = []
            claim_tokens = []
            if mode == 'clf':
              for item in data:
                  evidence = item['messages'][1]['content'].split('\n\n')[2]
                  evdc_counts.append(len(evidence.split('\n')))
                  evdc_tokens.append(len(nltk.word_tokenize(evidence)))
                  user_stub_tokens.append(len(nltk.word_tokenize(item['messages'][1]['content'].split('\n\n')[0])))
                  claim_tokens.append(len(nltk.word_tokenize(item['messages'][1]['content'].split('\n\n')[1])))
            elif mode == 'sentEx':
              for item in data:
                  evdc_counts.append(len(item['messages'][2]['content'].split('\n')))
                  evdc_tokens.append(len(nltk.word_tokenize(item['messages'][2]['content'])))
                  user_stub_tokens.append(len(nltk.word_tokenize(item['messages'][1]['content'].split('\n\n')[0])))
                  claim_tokens.append(len(nltk.word_tokenize(item['messages'][1]['content'].split('\n\n')[1])))

            # Calculate average, max, and min evidence sentence length
            avg_evdc_len = np.mean(evdc_counts)
            max_evdc_len = np.max(evdc_counts)
            min_evdc_len = np.min(evdc_counts)
            avg_evdc_tkns = np.mean(evdc_tokens)

            # Calculate average, max, and min claim token count
            avg_claim_len = np.mean(claim_tokens)
            max_claim_len = np.max(claim_tokens)
            min_claim_len = np.min(claim_tokens)

            item = data[0]
            sys_msg_tks, sys_msg_avg_wrd_len = tokenize_text(item['messages'][0]['content'])
            usr_msg_tks, usr_msg_avg_wrd_len = tokenize_text(item['messages'][1]['content'].split('\n\n')[0])
            # Append metadata to the DataFrame
            metadata.loc[len(metadata)] = [filename, ds_size, avg_evdc_len, max_evdc_len, min_evdc_len, avg_evdc_tkns, avg_claim_len, max_claim_len, min_claim_len, item['messages'][0]['content'], item['messages'][1]['content'].split('\n\n')[0], sys_msg_tks, sys_msg_avg_wrd_len, usr_msg_tks, usr_msg_avg_wrd_len]
    else:
        print(f"Error: {filename} is not a JSONL file.")
    # Save metadata to CSV
    save_path = out_path + f"{set_id}_metadata.csv"
    metadata.to_csv(save_path, index=False)
    print(f"Metadata saved to {save_path}")

# Collect metadata for the evidence extraction training data
sentEx_meta_paths = {
    "in_paths": [f"{fever_path}GPT_sentEx_paper_dev_train/prompt_v1_segmented_n200_03-16_001.jsonl", f"{fever_path}GPT_sentEx_paper_dev_valid/prompt_v1_segmented_n60_03-16_001.jsonl"],
    "out_paths": [f"{fever_path}GPT_meta/sentEx/train_", f"{fever_path}GPT_meta/sentEx/valid_"]
}
clf_meta_paths = {
    "in_paths": [f"{fever_path}GPT_clf_paper_dev_train/prompt_v1_segmented_n200_03-16_001.jsonl", f"{fever_path}GPT_clf_paper_dev_valid/prompt_v1_segmented_n60_03-16_001.jsonl"],
    "out_paths": [f"{fever_path}GPT_meta/clf/train_", f"{fever_path}GPT_meta/clf/valid_"]
}

for in_path, out_path in zip(sentEx_meta_paths['in_paths'], sentEx_meta_paths['out_paths']):
    collect_metadata(in_path, out_path, 'sentEx')
for in_path, out_path in zip(clf_meta_paths['in_paths'], clf_meta_paths['out_paths']):
    collect_metadata(in_path, out_path, 'clf')

Processing file: prompt_v1_segmented_n200_03-16_001.jsonl
Metadata saved to ./datasets/FEVER/GPT_meta/sentEx/train_001.jsonl_metadata.csv
Processing file: prompt_v1_segmented_n60_03-16_001.jsonl
Metadata saved to ./datasets/FEVER/GPT_meta/sentEx/valid_001.jsonl_metadata.csv
Processing file: prompt_v1_segmented_n200_03-16_001.jsonl
Metadata saved to ./datasets/FEVER/GPT_meta/clf/train_001.jsonl_metadata.csv
Processing file: prompt_v1_segmented_n60_03-16_001.jsonl
Metadata saved to ./datasets/FEVER/GPT_meta/clf/valid_001.jsonl_metadata.csv


In [ ]:
# Load the CSV training files and convert tokenized characters back
def replace_bracket_tokens(text):
    text = text.replace('-LRB-', '(')
    text = text.replace('-RRB-', ')')
    text = text.replace('-LSB-', '[')
    text = text.replace('-RSB-', ']')
    return text

def load_csv(filepath):
    """
    Load a CSV file and convert tokenized characters back to their original form.
    """
    df = pd.read_csv(filepath)
    # If there are three columns, replace the tokens in the first two columns
    if len(df.columns) == 3:
        df.iloc[:, 0] = df.iloc[:, 0].apply(replace_bracket_tokens)
        df.iloc[:, 1] = df.iloc[:, 1].apply(replace_bracket_tokens)
    # If there are four columns, replace the tokens in the first three columns
    elif len(df.columns) == 4:
        df.iloc[:, 0] = df.iloc[:, 0].apply(replace_bracket_tokens)
        df.iloc[:, 1] = df.iloc[:, 1].apply(replace_bracket_tokens)
        df.iloc[:, 2] = df.iloc[:, 2].apply(replace_bracket_tokens)

    return df

train_clf_path = f"{fever_path}tabular_clf_paper_dev_train/v1_segmented_n3468_03-29_001.csv"
valid_clf_path = f"{fever_path}tabular_clf_paper_dev_valid/v1_segmented_n1488_03-29_001.csv"
train_sentEx_path = f"{fever_path}tabular_sentEx_paper_dev_train/v1_segmented_n3468_03-29_001.csv"
valid_sentEx_path = f"{fever_path}tabular_sentEx_paper_dev_valid/v1_segmented_n1488_03-29_001.csv"

train_clf_df = load_csv(train_clf_path)
valid_clf_df = load_csv(valid_clf_path)
train_sentEx_df = load_csv(train_sentEx_path)
valid_sentEx_df = load_csv(valid_sentEx_path)

# Print info for each dataframe
print(f"Training data for classification:")
print(train_clf_df.info())
print(f"Training data for sentence extraction:")
print(train_sentEx_df.info())

print(f"Validation data for classification:")
print(valid_clf_df.info())
print(f"Validation data for sentence extraction:")
print(valid_sentEx_df.info())

Training data for classification:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3461 entries, 0 to 3460
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   claim               3461 non-null   object
 1   evidence_sentences  3461 non-null   object
 2   label               3461 non-null   object
dtypes: object(3)
memory usage: 81.2+ KB
None
Training data for sentence extraction:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3461 entries, 0 to 3460
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   claim               3461 non-null   object
 1   evidence_sentences  3461 non-null   object
 2   full_text           3461 non-null   object
 3   label               3461 non-null   object
dtypes: object(4)
memory usage: 108.3+ KB
None
Validation data for classification:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1478 ent

In [ ]:
pd.set_option('display.max_colwidth', None)
valid_clf_df.head(20)


,claim,evidence_sentences,label
0,The Late Show with Stephen Colbert is hosted by Stephen Colbert.,"Colbert has hosted The Late Show with Stephen Colbert , a late-night television talk show on CBS , since September 8 , 2015 .",SUPPORTS
1,Annette Badland played Bad Horse in The Sparticle Mystery.,"She has played Margaret Blaine in the BBC science fiction series Doctor Who , Doomsday Dora in The Sparticle Mystery , Birdie Henshall in the drama series Cutting It , Mavis in season 6 of Skins , Ursula Crowe in children 's science fiction/fantasy series Wizards vs Aliens , and Babe Smith in soap opera EastEnders .",REFUTES
2,Good over evil is signified spiritually through Diwali.,"It is currently held every two years.\nUntil 1989, the tournament was known as CONCACAF Championship.\nThe CONCACAF Gold Cup is North America's major tournament in senior men's soccer and determines the continental champion.\nFrom 1 to 2005, nations from other confederations have regularly joined the tournament as invitees.\nIn earlier editions, the continental championship was held in different countries, but since the inception of the Gold Cup in 1991, the United States are constant hosts or co-hosts.",NOT ENOUGH INFO
3,Danger UXB is a British television series.,"Danger UXB is a 1979 British ITV television series set during the Second World War developed by John Hawkesworth and starring Anthony Andrews as Lieutenant Brian Ash , an officer in the Royal Engineers .",SUPPORTS
4,Shadowhunters was not renewed for a third season in April 2017.,"In April 2017 , it was announced that the series had been renewed for a third season of 20 episodes .",REFUTES
5,Nestor Carbonell played Richard Alpert in Lost during his presidency.,"In 2 he starred in Sparsha, which gave him breakthrough.\nSudeepa is an Indian actor and director who primarily works as leading actor role in the Kannada film industry.\nSudeep made his debut in Thayavva (1997).\nHe has also worked in Hindi, Telugu and Tamil films.\nHe is also popularly known as Kiccha Sudeepa.",NOT ENOUGH INFO
6,There is a film that is about an ex-convict titled Swordfish (film).,"The film centers on Stanley Jobson , an ex-con and computer hacker who is targeted for recruitment into a bank robbery conspiracy because of his formidable hacking skills .",SUPPORTS
7,Match Point is a brand of radios.,"Match Point is a 2005 British-Luxembourgish psychological thriller film written and directed by Woody Allen and starring Jonathan Rhys Meyers , Scarlett Johansson , Emily Mortimer , Matthew Goode , Brian Cox , and Penelope Wilton .",REFUTES
8,Girl is related to G I R L.,"Underground music is music with practices perceived as outside, or somehow opposed to, mainstream popular music culture.\nThe term has been applied to artists in styles such as psychedelia, punk, alternative rock, electronica, industrial music, and wider strains of experimental music.\nUnderground music may be perceived as expressing sincerity and creative freedom in opposition to those practices deemed formulaic or market-driven.\nUnderground styles lack the commercial success of popular music movements, and may involve the use of avant-garde or abrasive approaches.\nNotions of individuality and non-conformity are also commonly deployed.",NOT ENOUGH INFO
9,Sidse Babett Knudsen does television work.,"Knudsen achieved international recognition for her leading role as fictional Danish Prime Minister Birgitte Nyborg in the Danish TV series Borgen .\nSidse Babett Knudsen ( [ ˈsisə b̥ab̥ɛd̥ ˈkʰnusn̩ ] ; born 22 November 1968 ) is a Danish actress who works in theatre , television , and film .",SUPPORTS


In [63]:
# Save all four dataframes back to the same directories
train_clf_df.to_csv(f"{fever_path}tabular_clf_paper_dev_train/v1_segmented_n3468_03-29_001.csv", index=False)
valid_clf_df.to_csv(f"{fever_path}tabular_clf_paper_dev_valid/v1_segmented_n1488_03-29_001.csv", index=False)
train_sentEx_df.to_csv(f"{fever_path}tabular_sentEx_paper_dev_train/v1_segmented_n3468_03-29_001.csv", index=False)
valid_sentEx_df.to_csv(f"{fever_path}tabular_sentEx_paper_dev_valid/v1_segmented_n1488_03-29_001.csv", index=False)

In [57]:
import spacy
# Import progress bar
from tqdm import tqdm
tqdm.pandas()

def calculate_syntactic_complexity(df, verbose=False):
    """
    Calculates syntactic complexity of text in the first column of a Pandas DataFrame, per the formula from Salman et al., 2023

    Args:
        df: Pandas DataFrame with text in the first column.

    Returns:
        Pandas DataFrame with an additional 'syntactic_complexity' column.
    """

    nlp = spacy.load("en_core_web_sm")  # Load a suitable spaCy model

    tw = 0.07  # Token weight
    vw = 0.3   # Verb weight
    cw = 0.4   # Conjunction weight

    sc_scores = []

    # Wrap the iterable with tqdm for progress bar
    for text in tqdm(df.iloc[:, 0], desc="Calculating syntactic complexity"):
        doc = nlp(text)
        tkn = len(doc)
        vrb = 0
        cnj = 0

        for token in doc:
            if token.pos_ == "VERB":
                vrb += 1
            elif token.pos_ in {"CONJ", "CCONJ", "SCONJ"}:  # Use a set for conjunction categories
                cnj += 1

        sc = tkn * tw + vrb * vw + cnj * cw
        sc_scores.append(sc)

    df['syntactic_complexity'] = sc_scores  # Add results to DataFrame
    return df

#train_clf_df = calculate_syntactic_complexity(train_clf_df, True)
valid_clf_df = calculate_syntactic_complexity(valid_clf_df)
train_sentEx_df = calculate_syntactic_complexity(train_sentEx_df)
valid_sentEx_df = calculate_syntactic_complexity(valid_sentEx_df)

Calculating syntactic complexity: 100%|██████████| 1482/1482 [00:11<00:00, 126.35it/s]


In [62]:
# Print info for each dataframe
print(f"Training data for classification:")
print(train_clf_df.info())
print(f"Training data for sentence extraction:")
print(train_sentEx_df.info())

print(f"Validation data for classification:")
print(valid_clf_df.info())
print(f"Validation data for sentence extraction:")
print(valid_sentEx_df.info())

Training data for classification:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3461 entries, 0 to 3460
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   claim                 3461 non-null   object 
 1   evidence_sentences    3461 non-null   object 
 2   label                 3461 non-null   object 
 3   syntactic_complexity  3461 non-null   float64
dtypes: float64(1), object(3)
memory usage: 108.3+ KB
None
Training data for sentence extraction:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3461 entries, 0 to 3460
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   claim                 3461 non-null   object 
 1   evidence_sentences    3461 non-null   object 
 2   full_text             3461 non-null   object 
 3   label                 3461 non-null   object 
 4   syntactic_complexity  3461 non-null   f

In [59]:
# Drop 4 samples from valid_sentEx_df randomly
valid_sentEx_df = valid_sentEx_df.drop(valid_sentEx_df.sample(n=4).index)

In [61]:
valid_sentEx_df.reset_index(drop=True, inplace=True)